In [1]:
import arcpy

# set initial workspace
arcpy.env.workspace = r'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb'

In [13]:
# create lists of the annual variable fields

attribute_names = ['resunits', 'homesteads', 'nonressf', 'pyr_market', 'pyr_taxes']

In [12]:
# For each variable of interest, transpose the table to pull out one row for each year for each hexagram. This will produce a separate table for each variable.

for name in attribute_names:
    in_table_transp = fr'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\{name}_hexbins'
    in_field_transp = f'{name}_2009_n 2009; {name}_2010_n 2010; {name}_2011_n 2011; {name}_2012_n 2012; {name}_2013_n 2013; {name}_2014_n 2014; {name}_2015_n 2015; {name}_2016_n 2016; {name}_2017_n 2017; {name}_2018_n 2018; {name}_2019_n 2019; {name}_2020_n 2020; {name}_2021_n 2021; {name}_2022_n 2022'
    out_table_transp = f'{name}_transp' 
    in_transposed_field_name = 'year' 
    in_value_field_name = f'{name}'
    attribute_fields = ['SHAPE', 'OBJECTID']
    
    arcpy.management.TransposeFields(in_table_transp, in_field_transp, out_table_transp, in_transposed_field_name, in_value_field_name, attribute_fields)

In [15]:
# For each new table, calculate a new field by copying 'year' into a new date type field. 

for name in attribute_names:
    in_table_date = f'{name}_transp'
    input_time_field = 'year'
    input_time_format = 'yyyy'
    output_time_field = 'yearDate'
    output_time_type = 'DATE'

    arcpy.management.ConvertTimeField(in_table_date, input_time_field, input_time_format, output_time_field, output_time_type)

In [16]:
# Space Time Cube requires a number variable to go into each bin, so we first need to convert the attribute field into a number type

for name in attribute_names:
    inFeatures_num = f'{name}_transp'
    field_name_num = f'{name}NUM'
    expression_num = f'!{name}!'

    arcpy.management.CalculateField(inFeatures_num, field_name_num, expression_num, "PYTHON3", field_type='FLOAT')

In [21]:
# create space time cube

for name in attribute_names:

    in_features_cube = f'{name}_transp'
    output_cube = fr'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\{name}_STCube.nc'
    location_id = 'OBJECTID_1'
    temporal_aggregation = 'NO_TEMPORAL_AGGREGATION'
    time_field = 'yearDate'
    varsOfInterest = f'{name}_N_SPACE_TIME_NEIGHBORS'    

    arcpy.stpm.CreateSpaceTimeCubeDefinedLocations(in_features_cube, output_cube, location_id, temporal_aggregation, time_field, time_step_interval= '1 Years', time_step_alignment='END_TIME', variables=varsOfInterest)

    # arcpy.stpm.CreateSpaceTimeCubeDefinedLocations(in_features, output_cube, location_id, temporal_aggregation, time_field, {time_step_interval}, {time_step_alignment}, {reference_time}, {variables}, {summary_fields}, {in_related_table}, {related_location_id})

In [24]:
# For each Space Time Cube, do an emerging hotspot/cold spot analysis.

for name in attribute_names:

    in_cube = fr'C:\Users\bryantch\Documents\ArcGIS\Projects\hexbin_parcel_analysis\hexbin_parcel_analysis.gdb\{name}_STCube.nc'
    name = name.upper
    analysis_variable = f'{name}_N_NONE_SPACE_TIME_NEIGHBORS' #getting error on this because it needs to be all caps. Ran all manually but needs to be fixed.
    output_features = f'{name}_STCube_EmergingHotSpotAnalysis'

    arcpy.stpm.EmergingHotSpotAnalysis(in_cube, analysis_variable, output_features, conceptualization_of_spatial_relationships='FIXED_DISTANCE', define_global_window='ENTIRE_CUBE')

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 110024: <built-in method upper of str object at 0x000002195734E770>NUM_NONE_SPACE_TIME_NEIGHBORS is not a variable in the Space Time Cube.
Failed to execute (EmergingHotSpotAnalysis).
